In [1]:
## Approach for 8 dice: Recursive method enumerate_rolls
## Tested out memoizing, but the check_possible function is already fast

In [1]:
import numpy as np
import numpy.random as rnd

In [2]:
#check_possible gets called over and over again with same N and M, therefore we use memoize class to speed up
def check_possible(N,M):
    #fail condition if sum of dice less than actual number of dice - also works when total hits 0 and total negative
    if M<N: 
        return False
    
    #even if we roll all sixes, can't reach sum - also works when total hits 0 and remaining sum too big
    if M>6*N: 
        return False
    
    return True

In [3]:
def enumerate_rolls(N,M,successful_rolls,current_roll):

    #this function only gets called in the first place if it's a viable roll, so we only need to check N==0
    if N==0:
        return successful_rolls + [current_roll]
        
    #enumerate possible rolls
    for roll in range(1,7):
        if check_possible(N-1,M-roll): #verifies if N and M combo can actually be reached
            successful_rolls = enumerate_rolls(N-1,M-roll, successful_rolls, current_roll + [roll])
    
    return successful_rolls #gets reached when recursion branches exhausted

In [4]:
## initial conditions - N=8, M=24
N = 8
M = 24
successful_rolls_init = []
current_roll_init = []

all_rolls = enumerate_rolls(N,M,successful_rolls_init,current_roll_init)

In [19]:
#fortunately, numpy allows easy conversion from list to numpy array:
products = np.prod(all_rolls, axis=1) 
print("Mean of dice products with number of dice N=8 and sum of faces M=24:", np.mean(products))
print("Standard deviation of dice products with number of dice N=8 and sum of faces M=24:", np.std(products))

Mean of dice products with number of dice N=8 and sum of faces M=24: 1859.93295417
Standard deviation of dice products with number of dice N=8 and sum of faces M=24: 855.069885347


In [5]:
## Part 2: Montecarlo simulation for large numbers of dice where enumerating is prohibitive
## Strategy: Generate 1 million rolls at a time, select rolls with correct sum, store their products.
## Repeat for niter iterations (I tried 50k max)

roll_products = np.array([]) #stores output
N = 50
M = 150
niter = 500 #how many times do we draw 1 million random sets of rolls?

log = open("/Users/Siwen/incubator/roll_trials.txt", "w")

for i in range(niter):
    print("Trial #" + str(i+1))
    random_rolls = rnd.randint(1,7,[1000000,N])
    sum_rolls = np.sum(random_rolls,1)
    successful_rolls = random_rolls[sum_rolls == M]
    new_products = np.prod(successful_rolls, axis = 1, dtype=np.uint64)
    print("New rolls found:", str(new_products.shape[0]))
    roll_products = np.append(roll_products, new_products, axis = 0)
    print("Total rolls found:", str(roll_products.shape[0]))
    
    if (i+1)%100 == 0:
        print("Mean roll product after " + str(i+1) + " million attempted rolls:", np.mean(roll_products), file=log)
        print("Standard deviation of roll product after " + str(i+1) + " million attempted rolls:", np.std(roll_products), file=log)

Trial #1
New rolls found: 3913
Total rolls found: 3913
Trial #2
New rolls found: 3903
Total rolls found: 7816
Trial #3
New rolls found: 3875
Total rolls found: 11691
Trial #4
New rolls found: 3786
Total rolls found: 15477
Trial #5
New rolls found: 3990
Total rolls found: 19467
Trial #6


KeyboardInterrupt: 

In [20]:
print('Mean roll product with number of dice N = 50 and roll sum M = 150:', np.mean(roll_products))
print('Standard deviation of roll product with number of dice N = 50 and roll sum M = 150:', \
      np.std(roll_products))

Mean roll product with number of dice N = 50 and roll sum M = 150: 9.03131053973e+18
Standard deviation of roll product with number of dice N = 50 and roll sum M = 150: 5.26244825337e+18
